In [2]:
import os
import re
import pickle

import nd2

import pandas as pd
import numpy as np

from tifffile import imsave

In [3]:
pathInfoFile=r'Z:\Wayne\20210716_sen_sig_IF\210716_info.csv'

pathData=r'Z:\Wayne\20210716_sen_sig_IF'
myWell = 'A3'

save_dir = r'D:\toDel'

In [21]:
def build_data_frame(pathInfoFile,pathData,myWell):
    
    # read in the file with the info about the eperiment
    myDataRounds = pd.read_csv(pathInfoFile)

    # create a list of subdirectories
    myFiles = os.listdir(pathData)

    myData=pd.DataFrame()

    k=0
    round_index = 0
    
    for i,myRoundInfo in myDataRounds.iterrows():

        mySubDirName = [x for x in myFiles if (f'Round_{myRoundInfo.myRound}_' in x)]

        mySubDir = os.path.join(pathData,mySubDirName[0])

        myFileName = [x for x in os.listdir(mySubDir) if f'Well{myWell}_Channel' in x][0]

        # get a handle to the file
        myIm = nd2.ND2File(os.path.join(mySubDir,myFileName))

        # through the channels in the file
        for j in range(myIm.sizes['C']):

            # get the channel name (OC)
            myChannel = myIm.metadata.channels[j].channel.name

            # translate the OC into the signal name
            mySignal = myRoundInfo[myChannel]

            if mySignal == mySignal: # if this channel was admitted (otherwise no entry in the info file)

                myData.loc[k,'dir'] = pathData
                myData.loc[k,'sub_dir'] = mySubDirName
                myData.loc[k,'file'] = myFileName
                myData.loc[k,'channel_in_file'] = j
                
                myData.loc[k,'nameRound'] = myRoundInfo.myRound      
                myData.loc[k,'well'] = myWell

                myData.loc[k,'signal'] = myRoundInfo[myChannel]
                myData.loc[k,'width'] = myIm.sizes['X']
                myData.loc[k,'height'] = myIm.sizes['Y']
                
                myData.loc[k,'alignRound'] = round_index

                k = k+1
                
        round_index = round_index + 1 
                
    # calculate min size
    width_min = np.min(myData.width)
    height_min = np.min(myData.height)
    
    myData['width_min'] = width_min
    myData['height_min'] = height_min
    
    return myData

def check_selection_to_align(myData):

    t  = myData.groupby(['nameRound']).sum()
    test1 = list(t.alignIm)==([1]*len(t))
    
    if test1:
        print('Data preparation passed tests.')
    else:
        print('Error - stop and report.')
        
def save_2align_files(myData,save_dir):

    saved = 0
    for ind,my_signal in myData.iterrows():

        if my_signal.alignIm == 1:

            # construct the path
            file_path = os.path.join(my_signal.dir,my_signal.sub_dir,my_signal.file)

            # get a handle to the nd2 file
            myIm = nd2.ND2File(file_path)

            # choose the right frame
            dask_im = myIm.to_dask()
            im = dask_im[my_signal.channel_in_file,:,:]
            
            # trim if needed
            my_well = my_signal.well
            my_round = str(int(my_signal.alignRound)).zfill(3)
            
            if ((my_signal.width_min < my_signal.width) | (my_signal.height_min < my_signal.height)):
                
                im = im[:my_signal.height_min,:my_signal.width_min]
                print(f'Image from well {my_well} from round{my_round} trimmed.')
                
            else:
                
                print(f'Image from well {my_well} from round{my_round} not trimmed.')

            # save the image
            imsave(os.path.join(save_dir,f'im2align_{my_well}_round_{my_round}.tif'),im.astype('uint16'))
            saved = saved + 1
    
    if saved == 0: 
        print(f'Saved {saved} images.')
    elif saved == 1:
        print(f'Saved {saved} image.')
    else:
        print(f'Saved {saved} images.')

In [22]:
# create a data frame
myData = build_data_frame(pathInfoFile,pathData,myWell)

# decide which frames will be used for alignment
myData['alignIm'] = (myData.signal=='DNA').astype(int)
check_selection_to_align(myData)

# save data frame
myData.to_pickle(os.path.join(pathData,f'df_{myWell}.pkl'))

# save channels to align
save_2align_files(myData,save_dir)

Data preparation passed tests.
Image A3 from round000 not trimmed.
Image A3 from round001 not trimmed.
Image A3 from round002 not trimmed.
Saved 3 images.


In [10]:
myData

,dir,sub_dir,file,channel_in_file,myRound,well,signal,width,height,alignRound,width_min,height_min,alignIm
0,Z:\Wayne\20210716_sen_sig_IF,20210716_Round_1_DNA_bGal,"WellA3_ChannelHoechst,AF488_Seq1202.nd2",0.0,1.0,A3,DNA,16589.0,16590.0,0.0,16589.0,16590.0,1
1,Z:\Wayne\20210716_sen_sig_IF,20210716_Round_1_DNA_bGal,"WellA3_ChannelHoechst,AF488_Seq1202.nd2",1.0,1.0,A3,bGal,16589.0,16590.0,0.0,16589.0,16590.0,0
2,Z:\Wayne\20210716_sen_sig_IF,20210719_Round_2_DNA_AF555_AF647,"WellA3_ChannelHoechst,AF555,AF647_Seq0801.nd2",0.0,2.0,A3,DNA,16589.0,16590.0,1.0,16589.0,16590.0,1
3,Z:\Wayne\20210716_sen_sig_IF,20210719_Round_2_DNA_AF555_AF647,"WellA3_ChannelHoechst,AF555,AF647_Seq0801.nd2",1.0,2.0,A3,anti_1,16589.0,16590.0,1.0,16589.0,16590.0,0
4,Z:\Wayne\20210716_sen_sig_IF,20210719_Round_2_DNA_AF555_AF647,"WellA3_ChannelHoechst,AF555,AF647_Seq0801.nd2",2.0,2.0,A3,anti_2,16589.0,16590.0,1.0,16589.0,16590.0,0
5,Z:\Wayne\20210716_sen_sig_IF,20210725_Round_3_DNA_CDK4,"WellA3_ChannelHoechst,AF647_Seq0400.nd2",0.0,3.0,A3,DNA,16589.0,16590.0,2.0,16589.0,16590.0,1
6,Z:\Wayne\20210716_sen_sig_IF,20210725_Round_3_DNA_CDK4,"WellA3_ChannelHoechst,AF647_Seq0400.nd2",1.0,3.0,A3,CDK4,16589.0,16590.0,2.0,16589.0,16590.0,0
